
first importing required **libraries**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding, LSTM
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


uploading dataset

In [4]:
df = pd.read_csv("IMDB.csv")
df

,#,rating,title,username,review-date,review-text,actions-helpful,Type
0,1,10,One of the finest TV series coming from India.,authentic_writer,15-May-20,I was hooked to it from the very first episode...,66 out of 112 found this helpful.,Positive
1,2,10,Fantastic,ansegal,15-May-20,Spell bound I was and for a change the police ...,63 out of 110 found this helpful.,Positive
2,3,10,Prime does it again,abhisheksarkar901,16-May-20,Avoid watching it if you are disassociated wit...,9 out of 13 found this helpful.,Positive
3,4,10,One of finest of 2020,aspnishanth-81860,15-May-20,When u start watch it...you will binge watch i...,40 out of 77 found this helpful.,Positive
4,5,9,Worth Binge Watching,rahulreeves,16-May-20,"Great story, content and justice done by the c...",7 out of 10 found this helpful.,Positive
...,...,...,...,...,...,...,...,...
370,371,9,Realistic,akjhacipet,16-May-20,Whole series looks very much real & story is c...,3 out of 6 found this helpful.,Positive
371,372,10,Quite hard hitting,KebbyPro,16-May-20,This series is not for the faint hearts. The m...,3 out of 6 found this helpful.,Positive
372,373,10,Mr Ahlawat You are Simply Awesome,dysondarryl,16-May-20,One of the best from Indian Series.... Raw Har...,3 out of 6 found this helpful.,Positive
373,374,10,You cant stop watching it,harmeet-mehta,16-May-20,"Undoubtedly, till May 2020, Paatal Lok is the ...",3 out of 6 found this helpful.,Positive


In [5]:
df.head()

,#,rating,title,username,review-date,review-text,actions-helpful,Type
0,1,10,One of the finest TV series coming from India.,authentic_writer,15-May-20,I was hooked to it from the very first episode...,66 out of 112 found this helpful.,Positive
1,2,10,Fantastic,ansegal,15-May-20,Spell bound I was and for a change the police ...,63 out of 110 found this helpful.,Positive
2,3,10,Prime does it again,abhisheksarkar901,16-May-20,Avoid watching it if you are disassociated wit...,9 out of 13 found this helpful.,Positive
3,4,10,One of finest of 2020,aspnishanth-81860,15-May-20,When u start watch it...you will binge watch i...,40 out of 77 found this helpful.,Positive
4,5,9,Worth Binge Watching,rahulreeves,16-May-20,"Great story, content and justice done by the c...",7 out of 10 found this helpful.,Positive


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   #                375 non-null    int64 
 1   rating           375 non-null    int64 
 2   title            375 non-null    object
 3   username         375 non-null    object
 4   review-date      375 non-null    object
 5   review-text      375 non-null    object
 6   actions-helpful  375 non-null    object
 7   Type             375 non-null    object
dtypes: int64(2), object(6)
memory usage: 23.6+ KB


checking the data is balanced or not

In [7]:
df["Type"].value_counts()

Positive    274
Negative    101
Name: Type, dtype: int64

In [8]:
df.head()

,#,rating,title,username,review-date,review-text,actions-helpful,Type
0,1,10,One of the finest TV series coming from India.,authentic_writer,15-May-20,I was hooked to it from the very first episode...,66 out of 112 found this helpful.,Positive
1,2,10,Fantastic,ansegal,15-May-20,Spell bound I was and for a change the police ...,63 out of 110 found this helpful.,Positive
2,3,10,Prime does it again,abhisheksarkar901,16-May-20,Avoid watching it if you are disassociated wit...,9 out of 13 found this helpful.,Positive
3,4,10,One of finest of 2020,aspnishanth-81860,15-May-20,When u start watch it...you will binge watch i...,40 out of 77 found this helpful.,Positive
4,5,9,Worth Binge Watching,rahulreeves,16-May-20,"Great story, content and justice done by the c...",7 out of 10 found this helpful.,Positive


checking null values

In [9]:
df.isnull().sum()

#                  0
rating             0
title              0
username           0
review-date        0
review-text        0
actions-helpful    0
Type               0
dtype: int64

creating a funtion to clean the reviews all at once

In [10]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

applying the function

In [11]:
df["clean_review"]=df["review-text"].apply(cleantext)

In [12]:
df.head()

,#,rating,title,username,review-date,review-text,actions-helpful,Type,clean_review
0,1,10,One of the finest TV series coming from India.,authentic_writer,15-May-20,I was hooked to it from the very first episode...,66 out of 112 found this helpful.,Positive,hooked first episode main character true repre...
1,2,10,Fantastic,ansegal,15-May-20,Spell bound I was and for a change the police ...,63 out of 110 found this helpful.,Positive,spell bound change police seems real courtysid...
2,3,10,Prime does it again,abhisheksarkar901,16-May-20,Avoid watching it if you are disassociated wit...,9 out of 13 found this helpful.,Positive,avoid watching disassociated gruesome reality ...
3,4,10,One of finest of 2020,aspnishanth-81860,15-May-20,When u start watch it...you will binge watch i...,40 out of 77 found this helpful.,Positive,u start watch binge watch start ask season wan...
4,5,9,Worth Binge Watching,rahulreeves,16-May-20,"Great story, content and justice done by the c...",7 out of 10 found this helpful.,Positive,great story content justice done character rea...


spliting the data into x and y

In [13]:
x = df["clean_review"]
y = df.Type.replace({'Negative':0,'Positive':1})

In [14]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)

now getting the sentence length

In [15]:
sentlen = []

for sent in df["clean_review"]:
  sentlen.append(len(word_tokenize(sent)))

df["SentLen"] = sentlen
df.head()

,#,rating,title,username,review-date,review-text,actions-helpful,Type,clean_review,SentLen
0,1,10,One of the finest TV series coming from India.,authentic_writer,15-May-20,I was hooked to it from the very first episode...,66 out of 112 found this helpful.,Positive,hooked first episode main character true repre...,27
1,2,10,Fantastic,ansegal,15-May-20,Spell bound I was and for a change the police ...,63 out of 110 found this helpful.,Positive,spell bound change police seems real courtysid...,35
2,3,10,Prime does it again,abhisheksarkar901,16-May-20,Avoid watching it if you are disassociated wit...,9 out of 13 found this helpful.,Positive,avoid watching disassociated gruesome reality ...,20
3,4,10,One of finest of 2020,aspnishanth-81860,15-May-20,When u start watch it...you will binge watch i...,40 out of 77 found this helpful.,Positive,u start watch binge watch start ask season wan...,10
4,5,9,Worth Binge Watching,rahulreeves,16-May-20,"Great story, content and justice done by the c...",7 out of 10 found this helpful.,Positive,great story content justice done character rea...,36


In [16]:
max(sentlen)

386

maximum length of a review is 386





In [17]:
np.quantile(sentlen, 0.95)

75.0

here the 95% percentile of the data has 75 length of there sentence

In [18]:
max_len = np.quantile(sentlen, 0.95)

giving token to every characters in the dataset as per there appearence

In [19]:
tok = Tokenizer(char_level=False, split=" ")
#char_level	if True, every character will be treated as a token.

tok.fit_on_texts(xtrain)
tok.index_word

{1: 'series',
 2: 'show',
 3: 'story',
 4: 'one',
 5: 'good',
 6: 'watch',
 7: 'character',
 8: 'jaideep',
 9: 'like',
 10: 'acting',
 11: 'time',
 12: 'best',
 13: 'well',
 14: 'ahlawat',
 15: 'lok',
 16: 'great',
 17: 'every',
 18: 'crime',
 19: 'actor',
 20: 'performance',
 21: 'direction',
 22: 'hindu',
 23: 'episode',
 24: 'thriller',
 25: 'watching',
 26: 'indian',
 27: 'content',
 28: 'must',
 29: 'make',
 30: 'got',
 31: 'go',
 32: 'people',
 33: 'season',
 34: 'real',
 35: 'paatal',
 36: 'script',
 37: 'india',
 38: 'amazing',
 39: 'also',
 40: 'web',
 41: 'could',
 42: 'sharma',
 43: 'banerjee',
 44: 'u',
 45: 'storyline',
 46: 'worth',
 47: 'really',
 48: 'brilliant',
 49: 'game',
 50: 'sacred',
 51: 'mirzapur',
 52: 'prime',
 53: 'always',
 54: 'cast',
 55: 'role',
 56: 'end',
 57: 'propaganda',
 58: 'abhishek',
 59: 'neeraj',
 60: 'drama',
 61: 'binge',
 62: 'man',
 63: 'dialogue',
 64: 'screenplay',
 65: 'right',
 66: 'police',
 67: 'get',
 68: 'kabi',
 69: 'better',
 70:

calculating total number of unique words

In [20]:
vocab_len = len(tok.index_word)
vocab_len

1662

applying the token numbers to the reviews in the dataset

In [21]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[90, 363, 240, 712, 241, 26, 2, 364, 713, 1],
 [13, 146, 300, 301, 166, 5, 10, 8, 14, 58, 43, 59, 68, 302, 303, 216, 60, 3],
 [38, 10, 714, 147, 133, 148, 11, 77, 189, 134],
 [149,
  242,
  19,
  487,
  13,
  304,
  365,
  488,
  1,
  489,
  4,
  150,
  490,
  2,
  150,
  69,
  715,
  716,
  100,
  135,
  190,
  717,
  366,
  718,
  719,
  9,
  91,
  491,
  720,
  101,
  53,
  721,
  492],
 [136,
  722,
  61,
  493,
  92,
  23,
  4,
  31,
  39,
  32,
  217,
  4,
  151,
  723,
  724,
  725,
  726,
  2],
 [137, 78, 727, 40, 1, 47, 28, 6, 1086, 167, 367, 44, 494, 368],
 [136,
  728,
  495,
  1087,
  191,
  18,
  24,
  1,
  79,
  1088,
  1089,
  1090,
  1091,
  70,
  1092,
  168,
  32,
  11,
  80,
  729,
  1,
  496,
  17,
  730,
  1093,
  169,
  1094,
  1095,
  1096,
  1,
  1097,
  731,
  1098,
  71,
  71,
  137,
  18,
  138,
  1099,
  115,
  1100,
  3,
  20,
  48,
  497,
  8,
  498,
  59,
  68,
  732,
  733,
  218,
  1101,
  58,
  43,
  305,
  170,
  45,
  499,
  149,
  243,
  1102,
  50

now applying the sequence as per the max length and Tokenizing it

In [22]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[  0,   0,   0, ..., 364, 713,   1],
       [  0,   0,   0, ..., 216,  60,   3],
       [  0,   0,   0, ...,  77, 189, 134],
       ...,
       [  0,   0,   0, ..., 364, 713,   1],
       [  0,   0,   0, ..., 827, 828, 829],
       [  0,   0,   0, ...,  36,   5,  10]], dtype=int32)

saving it in a variable to perform training and testing

In [23]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [24]:
vocab_len

1662

Building the RNN Model  

In [25]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,90,input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=50, epochs=30)

ypred = rnn.predict(seqmattest)

ypred = ypred>0.5


Epoch 1/30
6/6 [==============================] - 4s 33ms/step - loss: 0.6444
Epoch 2/30
6/6 [==============================] - 0s 33ms/step - loss: 0.4995
Epoch 3/30
6/6 [==============================] - 0s 33ms/step - loss: 0.3529
Epoch 4/30
6/6 [==============================] - 0s 38ms/step - loss: 0.2332
Epoch 5/30
6/6 [==============================] - 0s 31ms/step - loss: 0.1430
Epoch 6/30
6/6 [==============================] - 0s 34ms/step - loss: 0.0959
Epoch 7/30
6/6 [==============================] - 0s 32ms/step - loss: 0.0694
Epoch 8/30
6/6 [==============================] - 0s 33ms/step - loss: 0.0426
Epoch 9/30
6/6 [==============================] - 0s 35ms/step - loss: 0.0306
Epoch 10/30
6/6 [==============================] - 0s 32ms/step - loss: 0.0223
Epoch 11/30
6/6 [==============================] - 0s 32ms/step - loss: 0.0163
Epoch 12/30
6/6 [==============================] - 0s 33ms/step - loss: 0.0148
Epoch 13/30
6/6 [==============================] - 0s 34ms/st

checking the Accuracy

In [26]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.62      0.90      0.74        31
           1       0.96      0.79      0.87        82

    accuracy                           0.82       113
   macro avg       0.79      0.85      0.80       113
weighted avg       0.86      0.82      0.83       113



In [27]:
#=======================

now BUilding and applying the LSTM Model

In [28]:
lstm = Sequential()

lstm.add(Embedding(vocab_len+1,90,input_length=int(max_len), mask_zero=True))
lstm.add(LSTM(units=32, activation="tanh"))
lstm.add(Dense(units=32, activation="relu"))
lstm.add(Dropout(0.2))

lstm.add(Dense(units=1, activation="sigmoid"))

lstm.compile(optimizer="adam", loss="binary_crossentropy")

lstm.fit(seqmattrain, ytrain, batch_size=50, epochs=30)

ypred = lstm.predict(seqmattest)

ypred = ypred>0.5


Epoch 1/30
6/6 [==============================] - 6s 58ms/step - loss: 0.6873
Epoch 2/30
6/6 [==============================] - 0s 58ms/step - loss: 0.6621
Epoch 3/30
6/6 [==============================] - 0s 60ms/step - loss: 0.6200
Epoch 4/30
6/6 [==============================] - 0s 61ms/step - loss: 0.5298
Epoch 5/30
6/6 [==============================] - 1s 93ms/step - loss: 0.4133
Epoch 6/30
6/6 [==============================] - 1s 106ms/step - loss: 0.3191
Epoch 7/30
6/6 [==============================] - 1s 138ms/step - loss: 0.2529
Epoch 8/30
6/6 [==============================] - 1s 123ms/step - loss: 0.1941
Epoch 9/30
6/6 [==============================] - 1s 128ms/step - loss: 0.1424
Epoch 10/30
6/6 [==============================] - 1s 138ms/step - loss: 0.1065
Epoch 11/30
6/6 [==============================] - 1s 106ms/step - loss: 0.0777
Epoch 12/30
6/6 [==============================] - 1s 112ms/step - loss: 0.0516
Epoch 13/30
6/6 [==============================] - 1s 

accuracy after Lstm model

In [30]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        31
           1       1.00      0.96      0.98        82

    accuracy                           0.97       113
   macro avg       0.96      0.98      0.97       113
weighted avg       0.98      0.97      0.97       113



Hence The ACCURACY of the Model is better in LSTM compared to RNN